In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clear-subset/evaluate_Corpus_subset.json
/kaggle/input/clear-subset/test_queries_subset
/kaggle/input/clear-subset/str_queries_answers.json
/kaggle/input/clear-subset/evaluate_rel_doc_subset
/kaggle/input/clear-subset/reformulated_Corpus_dict_subset.json
/kaggle/input/clear-subset/Passage_dict_subset
/kaggle/input/clear-subset/evaluate_rel_doc_subset.json
/kaggle/input/clear-subset/df_train_filtered_subset_2023-11-15_20-11-30.json
/kaggle/input/clear-subset/df_train_filtered_subset_reformulated.json
/kaggle/input/clear-subset/test_rel_doc_subset.json
/kaggle/input/clear-subset/str_test_answers_subset.json
/kaggle/input/clear-subset/evaluate_queries_subset
/kaggle/input/clear-subset/df_train_filtered_subset_reformulated_v1.json
/kaggle/input/clear-subset/str_test_corpus_subset.json
/kaggle/input/clear-subset/evaluate_Corpus_subset
/kaggle/input/clear-subset/Passage_dict_subset.json
/kaggle/input/clear-subset/str_queries.json
/kaggle/input/clear-subset/test_Corpus_subset.js

In [2]:
def read_json_with_frozenset_column(path, frozenset_column_name):
  df = pd.read_json(path, orient='records')
  df[frozenset_column_name] = df[frozenset_column_name].apply(lambda x: frozenset(x))
  return df

In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=98d87cfed4bf66a46b8c240a9d8d31e5865755ba4a1f442cb90ab8c54565b416
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan  6 00:41:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
class TripletsDataset(IterableDataset):
    def __init__(self, model, queries, corpus, train_triplets):
        self.model = model
        self.queries = queries
        self.corpus = corpus
        self.train_triplets = train_triplets

    def __iter__(self):
        count = 0

        for triplet in self.train_triplets:
            qid, pos_id, neg_id = triplet
            qid = str(qid)
            pos_id = str(pos_id)
            neg_id = str(neg_id)
            query_text = self.corpus[qid]
            pos_text = self.corpus[pos_id]
            neg_text = self.corpus[neg_id]
            yield InputExample(texts=[query_text, pos_text, neg_text])

    def __len__(self):
        return len(train_triplets)

In [7]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout
train_batch_size = 64 #Increasing the train batch size improves the model performance, but requires more GPU memory

# The  model we want to fine-tune


model_name = 'distilroberta-base'
word_embedding_model = models.Transformer(model_name, max_seq_length=350)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# We construct the SentenceTransformer bi-encoder from scratch

# model = SentenceTransformer(model_name)

### Now we read the MS Marco dataset


# for p in [15]:
#   for n in [15]:
#     p=str(p)
#     n=str(n)

#reset model
model = SentenceTransformer(model_name)
data_folder = '/kaggle/input/clear-subset'
# os.makedirs(data_folder, exist_ok=True)
import json
model_save_path = '/kaggle/working/CLEAR/model/bi-encoder_distilroberta-base'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


df_train = read_json_with_frozenset_column('/kaggle/input/clear-subset/df_train_filtered_subset_reformulated_v1.json', 'set_answer')
corpus = {}
title_list = df_train['title'].to_list()

for idx, tmp in enumerate(title_list):
  corpus[str(idx)] = tmp

print('len corpus:', len(corpus))
print(corpus)


Passage_dict = {}
collection_filepath = os.path.join(data_folder, 'Passage_dict_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  Passage_dict = json.load(fIn)

evaluate_Corpus = {}
collection_filepath = os.path.join(data_folder, 'evaluate_Corpus_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  evaluate_Corpus = json.load(fIn)

evaluate_queries = {}
collection_filepath = os.path.join(data_folder, 'evaluate_queries_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  evaluate_queries = json.load(fIn)

evaluate_rel_doc = {}
collection_filepath = os.path.join(data_folder, 'evaluate_rel_doc_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  evaluate_rel_doc = json.load(fIn)



def get_triplets(Passage_dict):
    Triplets= []
    for k, v in Passage_dict.items():
        for x in v[0]:
            for y in v[1]:
                # query,same_appi,diff_api
                Triplets.append([k,x,y])
    return Triplets

train_triplets = get_triplets(Passage_dict)



print(evaluate_queries)

eval_queries = {}
for k in evaluate_queries:
  eval_queries[str(k)] = corpus[str(k)]
eval_Corpus = {}
for k in evaluate_Corpus:
  eval_Corpus[str(k)] = corpus[str(k)]

dev_queries =  {str(k):v for k,v in eval_queries.items()}
dev_corpus = {str(k):v for k,v in eval_Corpus.items()}
dev_rel_docs = {str(k):set(str(v) for v in v[0]) for k,v in evaluate_rel_doc.items()}



ir_evaluator = evaluation.InformationRetrievalEvaluator(dev_queries, corpus, dev_rel_docs, name='distilroberta-train_eval')

# For training the SentenceTransformer model, we need a dataset, a dataloader, and a loss used for training.
train_dataset = TripletsDataset(model=model, queries=corpus, corpus=corpus, train_triplets=train_triplets)
train_dataloader = DataLoader(train_dataset, shuffle=False, batch_size=train_batch_size)
train_loss = losses.MultipleNegativesRankingLoss(model=model)


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=ir_evaluator,
          epochs=50,
          warmup_steps=1000,
          output_path=model_save_path,
          evaluation_steps=5000,
          save_best_model=True,
          use_amp=True)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/534 [00:00<?, ?it/s]

Iteration:   0%|          | 0/534 [00:00<?, ?it/s]

Iteration:   0%|          | 0/534 [00:00<?, ?it/s]

Iteration:   0%|          | 0/534 [00:00<?, ?it/s]

Iteration:   0%|          | 0/534 [00:00<?, ?it/s]

In [8]:
model.evaluate(evaluator=ir_evaluator)

0.8818487564407067